In [3]:
from dotenv import load_dotenv
load_dotenv()
import os 
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec



In [11]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="reg", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [12]:
import json 
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Wilkins',
  'subject': 'Introduction to Computer Science',
  'stars': 4,
  'review': 'Dr. Wilkins is an excellent professor who makes even the most complex topics easy to understand. Her lectures are engaging and she is always available to help students during office hours.'},
 {'professor': 'Professor Michael Chen',
  'subject': 'Organic Chemistry',
  'stars': 3,
  'review': "Professor Chen is knowledgeable, but his lectures can be dry and difficult to follow at times. The course material is challenging, and he doesn't always provide clear explanations."},
 {'professor': 'Dr. Sarah Johnson',
  'subject': 'Introduction to Psychology',
  'stars': 5,
  'review': 'Dr. Johnson is an absolute gem. Her passion for the subject is contagious, and she creates a welcoming and inclusive classroom environment. I highly recommend this course.'},
 {'professor': 'Professor David Nguyen',
  'subject': 'Calculus I',
  'stars': 2,
  'review': 'I struggled greatly in this course

In [13]:
processed_data= []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata":{
            "review": review["review"],
            "subject": review ["subject"],
            "stars": review  ["stars"],
        }
    })


In [14]:
processed_data[0]

{'values': [-0.013717222958803177,
  -0.004423926584422588,
  -0.008146126754581928,
  0.04610646888613701,
  -0.026189645752310753,
  -0.012777520343661308,
  0.003734404221177101,
  0.004786993842571974,
  -0.004018145613372326,
  -0.015401366166770458,
  0.01681702211499214,
  -0.008426817134022713,
  0.003621517913416028,
  -0.006565717048943043,
  0.0167193915694952,
  0.03439069166779518,
  -0.022870175540447235,
  0.015901727601885796,
  -0.002913689473643899,
  0.026140829548239708,
  -0.004115777090191841,
  -0.01641429401934147,
  0.016878042370080948,
  -0.0132290655747056,
  -0.01967274397611618,
  -0.02283356338739395,
  0.024798396974802017,
  0.013253473676741123,
  -0.0012951426906511188,
  0.05306271091103554,
  0.0718567743897438,
  -0.007639663759618998,
  -0.01975817047059536,
  -0.014119952917098999,
  -0.04268936440348625,
  0.04061469808220863,
  0.01189273502677679,
  0.001367603545077145,
  0.005061582196503878,
  -0.009128543548285961,
  0.01556001789867878,
 

In [15]:
index = pc.Index('reg')
index.upsert(
    vectors=processed_data,
    namespace="nova1"
)

{'upserted_count': 45}

In [16]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'nova1': {'vector_count': 45}},
 'total_vector_count': 45}